<a href="https://colab.research.google.com/github/David-Delatorre/CECS-456-Project/blob/main/CECS456_CNN_Project_V5_David_Delatorre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing libraries
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from keras import backend as k

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

# Navigate to the path where the zip file is located
zip_file_path = '/content/drive/MyDrive/medical-mnist.zip'

import os
import zipfile

# Extract zip file from Google Drive to Colab
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')  # Extract to /content/dataset folder

# List the contents of the extracted folder
os.listdir('/content/dataset')

# Define the dataset directory
directory = "/content/dataset"

Mounted at /content/drive


# Loading the Dataset

In [9]:

train_ds, test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory, # Directory where the data is located
    labels='inferred', # labels are generated from the directory structure
    label_mode='categorical', #labels are encoded as a categorical vector
    class_names=None, #explicit list of class names in alphabetical order
    color_mode='grayscale', #images will be converted to have 1 channel
    batch_size=None, #Size of the batches of data.
    image_size=(48, 48), #Size to resize images to after they are read (h, w)
    shuffle=True, #Whether to shuffle the data.
    seed=42, #Optional random seed for shuffling and transformations.
    validation_split=0.2, #0<float<1, fraction of data to reserve for cv
    subset='both', #returns a tuple of two datasets (training & cv)
    interpolation='bilinear', #interpolation method used when resizing images
    follow_links=False, #Whether to visit subdirectories pointed to by symlinks
    crop_to_aspect_ratio=False, #resize the images with aspect ratio distortion
    pad_to_aspect_ratio=True, #resize the images with aspect ratio distortion
    data_format=None, #If None uses keras.config.image_data_format()
    verbose=True #Whether to display no. info on classes & no. of files found
)

Found 58954 files belonging to 6 classes.
Using 47164 files for training.
Using 11790 files for validation.


In [10]:
#Displays the 6 class names
print(train_ds.class_names)
#Displays element specifications
train_ds

['AbdomenCT', 'BreastMRI', 'CXR', 'ChestCT', 'Hand', 'HeadCT']


<_PrefetchDataset element_spec=(TensorSpec(shape=(48, 48, 1), dtype=tf.float32, name=None), TensorSpec(shape=(6,), dtype=tf.float32, name=None))>

In [22]:
#convert TensorFlow Datasets into NumPy format
train_ds_np = tfds.as_numpy(train_ds)
test_ds_np = tfds.as_numpy(test_ds)

n_train = len(train_ds)
n_test = len(test_ds)

print("Number of training samples:", n_train)
print("Number of testing samples:", n_test)

Number of training samples: 47164
Number of testing samples: 11790


In [33]:
img_height, img_width, channels = 48, 48, 1
num_classes = 6

# Preallocate arrays
X_train = np.zeros((n_train, img_height, img_width, channels))
y_train = np.zeros((n_train, num_classes))
X_test = np.zeros((n_test, img_height, img_width, channels))
y_test = np.zeros((n_test, num_classes))

# Fill the arrays
for i, ele in enumerate(train_ds_np):
    X_train[i] = ele[0]
    y_train[i] = ele[1]

for i, ele in enumerate(test_ds_np):
    X_test[i] = ele[0]
    y_test[i] = ele[1]

# Verify shapes
print("Shapes:")
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

# Accessing specific labels
print("Label of 421st train image:", y_train[421])
print("Label of 100th test image:", y_test[100])

#Expected Output:
# Shapes:
# (47164, 48, 48, 1) (47164, 6)
# (11790, 48, 48, 1) (11790, 6)
# Label of 421st train image: [0. 0. 1. 0. 0. 0.]
# Label of 100th test image: [0. 0. 0. 0. 1. 0.]

Shapes:
(47164, 48, 48, 1) (47164, 6)
(11790, 48, 48, 1) (11790, 6)
Label of 421st train image: [0. 0. 0. 1. 0. 0.]
Label of 100th test image: [0. 0. 0. 0. 1. 0.]


# Data Preprocessing:

In [34]:
#normalizing the image data by converting the pixel values from the range [0, 255] (which is typical for 8-bit grayscale images) to the range [0, 1]
X_train_normalized = X_train.astype('float32')
X_test_normalized = X_test.astype('float32')
X_train_normalized /= 255.0
X_test_normalized /= 255.0

# Model-1: CNN

In [35]:
import keras
from tensorflow.keras import datasets, layers, models, optimizers

In [36]:
my_model = models.Sequential([
    #2D Convolutional Layer with 32 filters each of size 3x3
    #Shape of the input images are 48x48 pixels with 1 channel
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    #Max Pooling Layer with a 2x2 pool size
    layers.MaxPooling2D((2, 2)),
    #2D Convolutional Layer with 64 filters each of size 3x3
    layers.Conv2D(64, (3, 3), activation='relu'),
    #Max Pooling Layer with a 2x2 pool size
    layers.MaxPooling2D((2, 2)),
    #2D Convolutional Layer with 128 filters each of size 3x3
    layers.Conv2D(128, (3, 3), activation='relu'),
    #This Layer flattens the 3D output of the last Conv2D layer into 1D array
    layers.Flatten(),
    #A fully connected (Dense) layer with 256 neurons
    layers.Dense(256, activation='relu'),
    #Dense layer with 6 neurons corresponding to the 6 classes
    layers.Dense(6, activation='softmax')
])

my_model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 46, 46, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 23, 23, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 21, 21, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 10, 10, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 8, 8, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │       2,097,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 6)                   │           1,542 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,191,622 (8.36 MB)

 Trainable params: 2,191,622 (8.36 MB)

 Non-trainable params: 0 (0.00 B)

In [37]:
my_model.compile(
    optimizer = 'adam', #the learning rate defaults to 0.001
    loss = 'categorical_crossentropy', #use this cross entropy function where are 2+ label classes in one-hot representation
    #loss_weights=None,
    metrics=['accuracy']
    #weighted_metrics=None,
    #run_eagerly=False,
    #steps_per_execution=1,
    #jit_compile='auto',
    #auto_scale_loss=True
)

In [38]:
#printing shapes and data types of training and testing sets
print(X_train_normalized.shape, y_train.shape, X_test.shape, y_test.shape)
print(X_train_normalized.dtype, y_train.dtype, X_test.dtype, y_test.dtype)

(47164, 48, 48, 1) (47164, 6) (11790, 48, 48, 1) (11790, 6)
float32 float64 float64 float64


In [39]:
history = my_model.fit(
    x=X_train_normalized,
    y=y_train,
    batch_size=None,
    epochs=10,
    #verbose=2,
    #callbacks=None,
    validation_split=0.2,
    #validation_data = (X_test,y_test)
    shuffle=True,
    #class_weight=None,
    #sample_weight=None,
    #initial_epoch=0,
    #steps_per_epoch=None,
    #validation_steps=None,
    #validation_batch_size=None,
    #validation_freq=1
)

Epoch 1/10
1180/1180 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9520 - loss: 0.1479 - val_accuracy: 0.9968 - val_loss: 0.0167
Epoch 2/10
1180/1180 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9977 - loss: 0.0093 - val_accuracy: 0.9971 - val_loss: 0.0162
Epoch 3/10
1180/1180 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9978 - loss: 0.0074 - val_accuracy: 0.9986 - val_loss: 0.0070
Epoch 4/10
1180/1180 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9991 - loss: 0.0027 - val_accuracy: 0.9984 - val_loss: 0.0078
Epoch 5/10
1180/1180 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9983 - loss: 0.0076 - val_accuracy: 0.9987 - val_loss: 0.0060
Epoch 6/10
1180/1180 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9995 - loss: 0.0024 - val_accuracy: 0.9981 - val_loss: 0.0124
Epoch 7/10
1180/1180 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.9994 - loss: 0.0015 - val_accuracy: 0.9979 - val_loss: 0.0116
Epoch 8/10
1180/1180 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9986 - loss: 0.0056

In [40]:
# Evaluate the model
test_loss, test_acc = my_model.evaluate(X_test_normalized, y_test)
print('Test accuracy:', test_acc)

369/369 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9964 - loss: 0.0293
Test accuracy: 0.9966921210289001


Printing some predictions:

In [41]:
predictions = my_model.predict(X_test_normalized[:10])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step


In [42]:
predicted_labels = predictions.argmax(axis=1)
actual_labels = y_test.argmax(axis=1)

print(f"Predicted Labels: {predicted_labels}")

print(f"Actual Labels: {actual_labels[:10]}")

Predicted Labels: [1 1 2 2 0 0 3 3 5 3]
Actual Labels: [1 1 2 2 0 0 3 3 5 3]
